# Politiques de résilience

<i>Si vous n'avez pas la possibilité d'éxécuter ce notebook (pas de .NET ou de Jupyter), vous pouvez utiliser [Binder](https://mybinder.org/v2/gh/mbaumanndev/prog-reseau-jupyter/master) pour l'éxécuter en ligne.</i>

Dans le cycle de vie de nos applications, il figure toujours l'imprévu : 

- La base de données ne répond plus
- Un service distant est KO
- Le service de cache est tombé
- etc

Pour palier, le mieux possible, à ces soucis, nous allons pouvoir utiliser des politiques de résilience.

En .NET (ainsi qu'en JavaScript), il existe des solutions clés en main pour définir et composer des politiques de résilience.

En .NET, cette solution s'appelle Polly (de même en JS, mais cette dernière est maintenue par Netflix).

![Logo de Polly](https://camo.githubusercontent.com/34b63e5aeab626ae9686f7243db578ca5dfdb21d/68747470733a2f2f7261772e6769746875622e636f6d2f4170702d764e6578742f506f6c6c792f6d61737465722f506f6c6c792d4c6f676f2e706e67)

Polly embarque plusieurs politiques de résilience pré-définies : https://github.com/App-vNext/Polly#resilience-policies

Avant d'aller plus loins, nous allons installer les paquets nuget nécessaires à son fonctionnement, et inclure l'ensemble des espaces de noms qui nous serons nécessaires pour éxécuter l'ensemble du notebook sans trop polluer le code. Pour celà, éxécutez la cellule suivante :

In [ ]:
#r "nuget:System.Text.Json, 4.7.1"
#r "nuget:System.Runtime.Caching, 4.7.0"
#r "nuget:Microsoft.Extensions.Caching.Memory, 3.0.0"
#r "nuget:Polly, 7.2.0"
#r "nuget:Polly.Caching.Memory, 3.0.2"

using System.Net.Http; // Espace dédié au HTTP, contient la class HttpClient
using System.Text.Json;
using System.Text.Json.Serialization;
using System.Collections.Generic;
using System.Linq;
using System.Threading;
using Microsoft.Extensions.Caching.Memory;
using Polly;
using Polly.Caching;
using Polly.Caching.Memory;
using Polly.Registry;
using Polly.Fallback;
using Polly.Retry;
using Polly.Bulkhead;
using Polly.Timeout;

/// <summary>
/// La classe WeatherForecast nous servira à récupérer nos JSON sous forme Objet
/// </summary>
public sealed class WeatherForecast
{
    [JsonPropertyName("date")]
    public DateTime Date { get; internal set; }
    
    [JsonPropertyName("temperatureC")]
    public int TemperatureC { get; set; }
    
    [JsonPropertyName("temperatureF")]
    public int TemperatureF => 32 + (int)(TemperatureC / 0.5556);
    
    [JsonPropertyName("summary")]
    public string Summary { get; set; }
}

Le paquet est désormais chargé dans notre notebook, nous allons pouvoir commencer à l'utiliser.

À la racine du dépôt se trouve une solution Visual Studio. Vous pouvez l'ouvrir avec Visual Studio Code équipé d'un module complémentaire, ou bien avec Visual Studio sur Windows et Mac.

Pour cette séance, la solution contient un unique projet : Une API Web qui simule la météo. À chaque appel, elle nous retourne des prévisions météo générées aléatoirement.

Nous allons, à travers ce notebook, établir une sandbox qui servira à émuler une partie d'une application, par exemple le site d'un journal local qui affiche un module de météo.

L'utilité de Polly ici est la suivante : si l'API de météo tombe en panne, mon module tombe en panne aussi. Dans le cas d'un module de météo, la criticité n'est pas énorme, mais sur un site de e-commerce, si le module de facturation tombe en panne, il faut établir d'autres stratégies selons les options à dispositions qui, elles, sont toujours fonctionnelles. C'est là que Polly intervient.

## Mise en application

Dans un premier temps, nous allons construire un simple client HTTP en .NET, avec l'API éteinte, pour observer le comportement du framework en cas de non-réponse de l'hôte intérrogé.

Avant d'éxécuter le code de la cellule, prennez le temps de démarrer l'API (dossier `apps/IutAmiens.ProgReseau.SampleApi/`). Si vous êtes sous Visual Studio, vous pouvez démarrer directement avec IIS Express, si vous souhaitez passer par la ligne de commande, faites `dotnet run`. Adaptez la variable `v_BaseUrl` avec les bons protocoles et le bon port.

> L'API embarque une spécification OpenAPI, une fois l'API lancée, rendez-vous dessus avec l'URL `/swagger/` pour disposer d'une sandbox.

In [ ]:
HttpClient v_Client = new HttpClient();
string v_BaseUrl = "https://localhost:44317/";
string v_ForecastUrl = $"{v_BaseUrl}weatherforecast";

Nous avons désormais instancié notre client HTTP ainsi que défini l'adresse de base que nous souhaitons joindre. Nous pouvons couper l'API avant d'éxécuter le bloc suivant.

In [ ]:
try
{
    HttpResponseMessage response = await v_Client.GetAsync(v_ForecastUrl);
    response.EnsureSuccessStatusCode();
    string responseBody = await response.Content.ReadAsStringAsync();

    Console.WriteLine(responseBody);
}  
catch(HttpRequestException e)
{
    Console.WriteLine("Une exception est survenue !");
    Console.WriteLine("Message : {0} ", e.Message);
}

Avec l'API coupée, on obtient une exception, si on lance l'API et qu'on joue de nouveau ce bloc, on voit qu'on obtient une réponse au format JSON. Si on le lance une seconde fois, le résultat est différent.

In [ ]:
try
{
    HttpResponseMessage response = await v_Client.GetAsync(v_ForecastUrl);
    response.EnsureSuccessStatusCode();
    string responseBody = await response.Content.ReadAsStringAsync();
    var result = JsonSerializer.Deserialize<IEnumerable<WeatherForecast>>(responseBody);
    
    display(result);
}  
catch(HttpRequestException e)
{
    Console.WriteLine("Une exception est survenue !");
    Console.WriteLine("Message : {0} ", e.Message);
}

try
{
    HttpResponseMessage response = await v_Client.GetAsync(v_ForecastUrl);
    response.EnsureSuccessStatusCode();
    string responseBody = await response.Content.ReadAsStringAsync();
    var result = JsonSerializer.Deserialize<IEnumerable<WeatherForecast>>(responseBody);
    
    display(result);
}  
catch(HttpRequestException e)
{
    Console.WriteLine("Une exception est survenue !");
    Console.WriteLine("Message : {0} ", e.Message);
}

Par contre, si l'API est de nouveau plus disponible, on doit s'attendre à avoir un nouveau plantage... C'est là que nous allons faire intervenir Polly !

Polly propose différentes politiques de résilience, qui sont détaillées ici : https://github.com/App-vNext/Polly#resilience-policies

Dans un premier temps, nous allons mettre en place un simple `Retry`. Nous allons pouvoir demander à Polly de ré-essayer d'éxécuter une action dans des conditions plus ou moins précises : en catchant sur `Exception`, on ré-essaye peu-importe l'exception levée, mais on peut également essayer d'attraper une exception précise, par exemple `HttpRequestException`. Exécutez le bloc suivant avec l'API éteinte.

In [ ]:
try
{
    var retryPolicy = Policy.Handle<Exception>()
        .RetryAsync(3, (exception, retryCount) =>
        {
            Console.WriteLine($"Exception {exception.GetType()} levée, tentative {retryCount}.");
        });
    
    string responseBody = await retryPolicy.ExecuteAsync(async () => {
        HttpResponseMessage response = await v_Client.GetAsync(v_ForecastUrl);
        response.EnsureSuccessStatusCode();
        return await response.Content.ReadAsStringAsync();
    });
    
    Console.WriteLine(responseBody);
}
catch (Exception e)
{
    Console.WriteLine("Une exception est survenue !");
    Console.WriteLine("Message : {0} ", e.Message);
}

Polly va tenter de joindre l'API trois fois avant de laisser tomber. Polly peut également re-tenter des actions selon le résultat retourné par les appels/méthodes que l'on y éxécute (il faut garder à l'esprit qu'on utilise Polly pour de la résilience réseau, mais on pourrait s'en servir pour de la résilience applicative avec des erreurs métier !).
Relançons notre API et éxécutons le code suivant. Cette fois, on va vérifier que la température du premier jour est d'au moins 10 degrés celcius avant de dire que le résultat est valable. De même, si le réseau tombe, on ré-essaiera de faire des appels jusqu'à avoir un premier jour au dessus de 10°C.

In [ ]:
try
{
    var retryPolicy = Policy.Handle<HttpRequestException>()
        .OrResult<IEnumerable<WeatherForecast>>(r => r.FirstOrDefault()?.TemperatureC < 10)
        .RetryAsync(15, (exception, retryCount) =>
        {
            Console.WriteLine($"Exception {exception.GetType()} levée, tentative {retryCount}.");
        });
    
    IEnumerable<WeatherForecast> response = await retryPolicy.ExecuteAsync(async () => {
        Task.Delay(1000);
        HttpResponseMessage response = await v_Client.GetAsync(v_ForecastUrl);
        response.EnsureSuccessStatusCode();
        string result = await response.Content.ReadAsStringAsync();
        return JsonSerializer.Deserialize<IEnumerable<WeatherForecast>>(result);
    });
    
    display(response);
}
catch (Exception e)
{
    Console.WriteLine("Une exception est survenue !");
    Console.WriteLine("Message : {0} ", e.Message);
}

Polly propose également une méthode pour attendre un peu avant de ré-essayer. Celle-ci s'avère pratique pour éviter de tenter d'appeler un hôte de façon continue alors que celui-ci est peut-être hors service. Avec l'API éteinte, éxécutez le bloc de code suivant. Nous remarquons que l'attente est graduelle. En effet, celle-ci est entièrement configurable et paramétrable selon le nombre de tentatives effectuées. Ici, chaque essai 

In [ ]:
try
{
    var retryPolicy = Policy.Handle<HttpRequestException>()
        .WaitAndRetryAsync(15,
            sleepDurationProvider: (retryCount) => TimeSpan.FromSeconds(retryCount),
            onRetry: (exception, delay, retryCount, ctx) =>
        {
            Console.WriteLine($"Exception {exception.GetType()} levée, tentative {retryCount}.");
            Console.WriteLine($"Attente de {delay.Seconds} seconde(s) avant un nouvel éssai.");
        });
    
    IEnumerable<WeatherForecast> response = await retryPolicy.ExecuteAsync(async () => {
        HttpResponseMessage response = await v_Client.GetAsync(v_ForecastUrl);
        response.EnsureSuccessStatusCode();
        string result = await response.Content.ReadAsStringAsync();
        return JsonSerializer.Deserialize<IEnumerable<WeatherForecast>>(result);
    });
    
    display(response);
}
catch (Exception e)
{
    Console.WriteLine("Une exception est survenue !");
    Console.WriteLine("Message : {0} ", e.Message);
}

Dans les autres possibilité que nous offre Polly, nous pouvons mettre en placce un fallback. Dans notre cas, si notre API ne répond pas, nous allons interroger une API située sur Azure.

In [ ]:
try
{

    var fallbackPolicy = Policy<IEnumerable<WeatherForecast>>.Handle<HttpRequestException>()
        .FallbackAsync<IEnumerable<WeatherForecast>>(async (ct) => {
            HttpResponseMessage response = await v_Client.GetAsync("https://lprgi-weather-forecast.azurewebsites.net/weatherforecast");
            response.EnsureSuccessStatusCode();
            string result = await response.Content.ReadAsStringAsync();
            return JsonSerializer.Deserialize<IEnumerable<WeatherForecast>>(result);
        },
        (exception) => {
            Console.WriteLine($"Exception {exception.GetType()} levée, appel du fallback.");
            return Task.CompletedTask;
        });
    
    IEnumerable<WeatherForecast> response = await fallbackPolicy.ExecuteAsync(async () => {
        HttpResponseMessage response = await v_Client.GetAsync(v_ForecastUrl);
        response.EnsureSuccessStatusCode();
        string result = await response.Content.ReadAsStringAsync();
        return JsonSerializer.Deserialize<IEnumerable<WeatherForecast>>(result);
    });
    
    display(response);
}
catch (Exception e)
{
    Console.WriteLine("Une exception est survenue !");
    Console.WriteLine("Message : {0} ", e.Message);
}

Un autre élément puissant que nous met à disposition Polly est le Circuit Breaker : Au bout d'un nombre d'événements que l'on choisit, Polly va de lui-même arrêter d'essayer d'éxécuter les actions qu'on lui demande pendant un temps que l'on défini. Dans l'exemple suivant, avec l'API éteinte, on va essayer d'appeler 3 fois notre API avec une politique WaitAndRetry avant de couper le circuit pendant 10 secondes, et, dans ce laps de temps de 10 secondes, nous allons de nouveau tenter de faire appel à notre API. Pour celà, on va utiliser une autre propriété de Polly qui permet de coupler les différentes politiques de résilience.

In [ ]:
var retryPolicy = Policy.Handle<HttpRequestException>()
    .WaitAndRetryAsync(3,
        sleepDurationProvider: (retryCount) => TimeSpan.FromSeconds(retryCount),
        onRetry: (exception, delay, retryCount, ctx) =>
    {
        Console.WriteLine($"Exception {exception.GetType()} levée, tentative {retryCount}.");
        Console.WriteLine($"Attente de {delay.Seconds} seconde(s) avant un nouvel éssai.");
    });

var circuitBreakerPolicy = Policy.Handle<HttpRequestException>()
    .CircuitBreakerAsync(
        exceptionsAllowedBeforeBreaking: 3,
        durationOfBreak: TimeSpan.FromSeconds(10),
        onBreak: (exception, timespan) => {
            Console.WriteLine($"Exception {exception.GetType()} levée, coupure du circuit pour {timespan.TotalSeconds} secondes.");
        },
        onReset: () => Console.WriteLine("Circuit fermé, retour à la normale"),
        onHalfOpen: () => Console.WriteLine("Circuit à moitié ouvert, en cas de nouvelle défaillance"));

var policies = Policy.WrapAsync(retryPolicy, circuitBreakerPolicy);

Console.WriteLine("Premier appel");

try
{
    IEnumerable<WeatherForecast> response = await policies.ExecuteAsync(async () => {
        HttpResponseMessage response = await v_Client.GetAsync(v_ForecastUrl);
        response.EnsureSuccessStatusCode();
        string result = await response.Content.ReadAsStringAsync();
        return JsonSerializer.Deserialize<IEnumerable<WeatherForecast>>(result);
    });
    
    display(response);
}
catch (Exception e)
{
    Console.WriteLine("Une exception est survenue !");
    Console.WriteLine("Message : {0} ", e.Message);
}

Console.WriteLine("Attente de 6 secondes");
Thread.Sleep(6000);

Console.WriteLine("Second appel");

try
{
    IEnumerable<WeatherForecast> response = await policies.ExecuteAsync(async () => {
        HttpResponseMessage response = await v_Client.GetAsync(v_ForecastUrl);
        response.EnsureSuccessStatusCode();
        string result = await response.Content.ReadAsStringAsync();
        return JsonSerializer.Deserialize<IEnumerable<WeatherForecast>>(result);
    });
    
    display(response);
}
catch (Exception e)
{
    Console.WriteLine("Une exception est survenue !");
    Console.WriteLine("Message : {0} ", e.Message);
}

Console.WriteLine("Attente de 6 secondes");
Thread.Sleep(6000);

Console.WriteLine("Troisième appel");

try
{
    IEnumerable<WeatherForecast> response = await policies.ExecuteAsync(async () => {
        HttpResponseMessage response = await v_Client.GetAsync(v_ForecastUrl);
        response.EnsureSuccessStatusCode();
        string result = await response.Content.ReadAsStringAsync();
        return JsonSerializer.Deserialize<IEnumerable<WeatherForecast>>(result);
    });
    
    display(response);
}
catch (Exception e)
{
    Console.WriteLine("Une exception est survenue !");
    Console.WriteLine("Message : {0} ", e.Message);
}

Console.WriteLine("Attente de 11 secondes");
Thread.Sleep(11000);

Console.WriteLine("Quatrième appel (API Azure)");

try
{
    IEnumerable<WeatherForecast> response = await policies.ExecuteAsync(async () => {
        HttpResponseMessage response = await v_Client.GetAsync("https://lprgi-weather-forecast.azurewebsites.net/weatherforecast");
        response.EnsureSuccessStatusCode();
        string result = await response.Content.ReadAsStringAsync();
        return JsonSerializer.Deserialize<IEnumerable<WeatherForecast>>(result);
    });
    
    display(response);
}
catch (Exception e)
{
    Console.WriteLine("Une exception est survenue !");
    Console.WriteLine("Message : {0} ", e.Message);
}

Il faut noter que Polly propose un CircuitBreaker avancé qui, en plus du nombre d'erreurs successives, peut-être configuré selon le taux d'échec des actions exécutées.

Enfin, la dernière politique que nous allons voir pour ce cours est le cache. En effet, Polly peut cacher des données pour une durée déterminée. Avant d'éxécuter le bloc de code suivant, pensez à démarrer votre API.

Avec Polly, nous pouvons configurer la durée du cache. Dans le code ci-dessous, le cache expirera au bout de 30 secondes. Une fois votre premier appel réussi, coupez l'API pour constater que le cache est bien appelé.

In [ ]:
var memoryCache = new MemoryCache(new MemoryCacheOptions());
var memoryCacheProvider = new MemoryCacheProvider(memoryCache);

var cachePolicy = Policy.CacheAsync<IEnumerable<WeatherForecast>>(memoryCacheProvider, TimeSpan.FromSeconds(30));
var policyExecutionContext = new Context("CacheDemo");

Console.WriteLine("Premier appel");

try
{
    IEnumerable<WeatherForecast> response = await cachePolicy.ExecuteAsync(async (context) => {
        HttpResponseMessage response = await v_Client.GetAsync(v_ForecastUrl);
        response.EnsureSuccessStatusCode();
        string result = await response.Content.ReadAsStringAsync();
        return JsonSerializer.Deserialize<IEnumerable<WeatherForecast>>(result);
    }, policyExecutionContext);
    
    display(response);
}
catch (Exception e)
{
    Console.WriteLine("Une exception est survenue !");
    Console.WriteLine("Message : {0} ", e.Message);
}

Console.WriteLine("Attente de 20 secondes");
Thread.Sleep(20000);

Console.WriteLine("Second appel");

try
{
    IEnumerable<WeatherForecast> response = await cachePolicy.ExecuteAsync(async (context) => {
        HttpResponseMessage response = await v_Client.GetAsync(v_ForecastUrl);
        response.EnsureSuccessStatusCode();
        string result = await response.Content.ReadAsStringAsync();
        return JsonSerializer.Deserialize<IEnumerable<WeatherForecast>>(result);
    }, policyExecutionContext);
    
    display(response);
}
catch (Exception e)
{
    Console.WriteLine("Une exception est survenue !");
    Console.WriteLine("Message : {0} ", e.Message);
}


On peut évidement faire cohabiter le cache avec les autres politiques mises à disposition. Avant de lancer le bloc suivant, allumez votre API, et éteignez là une fois qu'un appel a réussi. N'hésitez-pas à la rallumer après quelques appels échoués une fois le CircuitBreaker activé.

In [ ]:
var retryPolicy = Policy.Handle<HttpRequestException>()
    .WaitAndRetryAsync(3,
        sleepDurationProvider: (retryCount) => TimeSpan.FromSeconds(retryCount),
        onRetry: (exception, delay, retryCount, ctx) =>
    {
        Console.WriteLine($"Exception {exception.GetType()} levée, tentative {retryCount}.");
        Console.WriteLine($"Attente de {delay.Seconds} seconde(s) avant un nouvel éssai.");
    });

var circuitBreakerPolicy = Policy.Handle<Exception>()
    .CircuitBreakerAsync(
        exceptionsAllowedBeforeBreaking: 3,
        durationOfBreak: TimeSpan.FromSeconds(15),
        onBreak: (exception, timespan) => {
            Console.WriteLine($"Exception {exception.GetType()} levée, coupure du circuit pour {timespan.TotalSeconds} secondes.");
        },
        onReset: () => Console.WriteLine("Circuit fermé, retour à la normale"),
        onHalfOpen: () => Console.WriteLine("Circuit à moitié ouvert, en cas de nouvelle défaillance"));

var memoryCache = new MemoryCache(new MemoryCacheOptions());
var memoryCacheProvider = new MemoryCacheProvider(memoryCache);

var cachePolicy = Policy.CacheAsync<IEnumerable<WeatherForecast>>(memoryCacheProvider, TimeSpan.FromSeconds(30));
var policyExecutionContext = new Context("CacheDemo");

var policies = cachePolicy.WrapAsync(retryPolicy).WrapAsync(circuitBreakerPolicy);

for (int i = 0; i < 10; i++) {
    Console.WriteLine($"Appel n°{i}");

    try
    {
        IEnumerable<WeatherForecast> response = await policies.ExecuteAsync(async (context) => {
            HttpResponseMessage response = await v_Client.GetAsync(v_ForecastUrl);
            response.EnsureSuccessStatusCode();
            string result = await response.Content.ReadAsStringAsync();
            return JsonSerializer.Deserialize<IEnumerable<WeatherForecast>>(result);
        }, policyExecutionContext);

        display(response);
    }
    catch (Exception e)
    {
        Console.WriteLine("Une exception est survenue !");
        Console.WriteLine("Message : {0} ", e.Message);
    }
    
    Console.WriteLine("Attente de 10 secondes");
    Thread.Sleep(10000);
}

## À vous de jouer

Envoyez-moi un document d'ici le prochain cours. Vous y expliquerez, succintement, l'intérêt d'une librairie telle que Polly dans le cadre de l'élaboration d'une architecture SOA.
Vous y expliquerez aussi l'intérêt de la politique de résilience `Bulk Head Isolation` dans le cadre de ce genre d'architecture.